In [13]:
import pandas as pd
import numpy as np


# Определение ставок VAT и duty
vat_rates = {
    'DE': 0.19,            # Germany
    'PT': 0.23,            # Portugal (mainland, Madeira, Azores)
    'ES': 0.21,            # Spain (except Canary Islands)
    'IE': 0.23,            # Ireland
    'SE': 0.25,            # Sweden
    'NL': 0.21,            # Netherlands
    'DK': 0.25,            # Denmark (mainland only)
    'FI': 0.255,           # Finland (except Åland Islands, raised in 2024)
    'IT': 0.22,            # Italy (except Livigno, Campione d’Italia, Lake Lugano waters)
    'AT': 0.20,            # Austria
    'BE': 0.21,            # Belgium
    'EE': 0.22             # Estonia (until July); use 0.24 for returns post-July 2025
}

In [14]:
df = pd.read_csv("TED BAKER DUTY CLAIM BACK Jul-Sep v2.csv")

df = df[~df['Consignee Country'].isin(['IC', 'CH'])] # exclude all the non-eu countries

# convert placeholder strings to real missing values, then fill from Parcel ID
df['MRN'] = df['MRN'].replace(['#N/A', 'N/A', 'NA', 'na', ''], pd.NA)
df.loc[df['MRN'].isna(), 'MRN'] = df.loc[df['MRN'].isna(), 'Parcel ID']


# Ensure correct consignment grouping
df['Consignment Value'] = df.groupby('MRN')['Line Item Unit Price'].transform('sum')

#df = df[df['Consignment Value'] <= 150]

df = df[df['Line Item Quantity Returned'] > 0] # оставляем только те товары, которые вернули

In [15]:
df['VAT Rate'] = df['Consignee Country'].map(vat_rates)

df['VAT Returned'] = df['Line Item Quantity Returned'] * df['Line Item Unit Price'] * df['VAT Rate']

In [16]:
# group and sum specific columns
df = df.groupby(['Consignee Country', 'VAT Rate'], as_index=False).agg({
    'VAT Returned': 'sum'
})

# apply 0.3% for IE, otherwise 0.2%
df['DR Revenue'] = df['VAT Returned'] * np.where(df['Consignee Country'] == 'IE', 0.3, 0.2)

print(df[['Consignee Country','VAT Rate', 'VAT Returned' ,'DR Revenue']])

print(f'\nOUR REVENUE: {df['DR Revenue'].sum()}')

  Consignee Country  VAT Rate  VAT Returned  DR Revenue
0                AT     0.200       331.000     66.2000
1                DE     0.190     18230.481   3646.0962
2                DK     0.250       297.250     59.4500
3                ES     0.210      2562.630    512.5260
4                FI     0.255      1378.530    275.7060
5                IE     0.230     21664.620   6499.3860
6                IT     0.220       180.400     36.0800
7                NL     0.210      5833.590   1166.7180
8                PT     0.230       856.750    171.3500
9                SE     0.250       438.250     87.6500

OUR REVENUE: 12521.162199999999
